Carrega os dados do ElasticSearch

In [75]:
import elasticsearch
import pandas as pd

INDEX_NAME = "consultas"
SCROLL = "1m"

In [76]:
entries = []
# Busca todos os registros no ElasticSearch
es = elasticsearch.Elasticsearch(hosts=["localhost"])
query = {"query" : {"match_all" : {}}}

records = es.search(index=INDEX_NAME, body=query, scroll=SCROLL)
scroll_id = records["_scroll_id"]
entries += records["hits"]["hits"]

records = es.scroll(scroll=SCROLL, scroll_id=scroll_id)
while (records["hits"]["hits"]):
    entries += records["hits"]["hits"]
    records = es.scroll(scroll=SCROLL, scroll_id=scroll_id)

print("Total entries: ", len(entries))


# DataFrame para guardar os registros
data = pd.DataFrame(data=[e["_source"] for e in entries])

Total entries:  7608


In [77]:
# Mostre algumas entradas
data

,id,descricao,timestamp
0,406,dor de cabeça coriza enjoo,2020-04-05T23:36:48.879229
1,407,perda de olfato manchas na pele perda de cabelo,2020-04-05T23:36:48.895012
2,408,falta de ar convulsoes dor na garganta,2020-04-05T23:36:48.908735
3,409,perda de olfato febre inconciencia,2020-04-05T23:36:48.924186
4,410,manchas na pele diarreia dor na garganta,2020-04-05T23:36:48.936006
...,...,...,...
7603,7603,incontinencia urinaria dor na garganta enjoo,2020-04-05T23:38:46.422974
7604,7604,dor abdominal diarreia febre,2020-04-05T23:38:46.454579
7605,7605,dor na garganta inconciencia incontinencia uri...,2020-04-05T23:38:46.485609
7606,7606,dor abdominal perda de cabelo dor de barriga,2020-04-05T23:38:46.516511


In [78]:
# remoção de algumas palavras como "de", "da", "na", "no", etc. Remove tbm palavras duplicadas
def remove_palavras_indesejadas(descricao):
    def eh_palavra_indesejada(palavra):
        return not(palavra.strip() == "" or palavra.strip() in ["da", "de", "do", "na", "no"])
    # filtra palavras indesejadas e duplicadas
    return " ".join(filter(eh_palavra_indesejada, list(set(descricao.split(" ")))))
    
    
data["descricao"] = data["descricao"].map(remove_palavras_indesejadas)

In [79]:
data

,id,descricao,timestamp
0,406,enjoo cabeça coriza dor,2020-04-05T23:36:48.879229
1,407,olfato perda cabelo manchas pele,2020-04-05T23:36:48.895012
2,408,ar garganta convulsoes dor falta,2020-04-05T23:36:48.908735
3,409,inconciencia olfato perda febre,2020-04-05T23:36:48.924186
4,410,garganta diarreia dor manchas pele,2020-04-05T23:36:48.936006
...,...,...,...
7603,7603,enjoo garganta incontinencia dor urinaria,2020-04-05T23:38:46.422974
7604,7604,diarreia febre abdominal dor,2020-04-05T23:38:46.454579
7605,7605,garganta incontinencia dor urinaria inconciencia,2020-04-05T23:38:46.485609
7606,7606,barriga abdominal perda dor cabelo,2020-04-05T23:38:46.516511


In [80]:
# Vamos pegar todos os sintomas descritos
sintomas = set()
for descricao in data["descricao"].unique():
    sintomas.update(descricao.split(" "))
    
sintomas

{'abdominal',
 'ar',
 'barriga',
 'cabelo',
 'cabeça',
 'convulsoes',
 'coriza',
 'diarreia',
 'dor',
 'enjoo',
 'falta',
 'febre',
 'garganta',
 'inconciencia',
 'incontinencia',
 'labirintite',
 'manchas',
 'muscular',
 'olfato',
 'pele',
 'perda',
 'tontura',
 'urinaria'}